# Suite: User
Using Library Python Selenium to test the web site

In [1]:
# ####### DEBUG ONLY #######
# ####### Load the "autoreload" extension #######
%load_ext autoreload
# ####### always reload modules marked with "%import" #######
%autoreload 2

import os
import sys


sys.path.append("../config")
sys.path.append("../util")
sys.path.append("../tests")

In [3]:
# Importing packages
from config import config
from datetime import datetime, timedelta
import json
import logging
import logging.config
import pathlib
from pprint import pprint
# from web_driver import WebDriver
import time
import fx_test
from fx_test import TestEngine

from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.common.keys import Keys

import unittest
from ui_helper import UIHelper

## Variables Declaration

In [4]:
log_config = "../config/logging.conf"

# # Create folder "logs"
# pathlib.Path('../logs').mkdir(exist_ok=True)  
# logging.log_file = datetime.now().strftime('../logs/log_%Y%m%d_%H%M%S.log')
logging.config.fileConfig(log_config, disable_existing_loggers=False)
logger = logging.getLogger(__name__)

suite_name = "suite_user"
suite_case = "suite_01"
logger.info("Test Suite: {}".format(suite_name))
logger.info("Suite Case: {}".format(suite_case))

[INFO] Test Suite: suite_user
[INFO] Suite Case: suite_01


## Functions Definition

### Main Function Begin
def main(args):

In [5]:
mode = config["web"]["mode"]
base_url = config["web"]["base_url"]
timeout = config["web"]["timeout"]
user_name = config["web"]["user_name"]
password = config["web"]["password"]

if mode == "windows":
    driver_path = "../../../library/drivers/chrome/win/chromedriver.exe"
else:
    driver_path = "../../../library/drivers/chrome/mac/chromedriver"
    
logger.info("Mode: {}".format(mode))
logger.info("Base Url: {}".format(base_url))
logger.info("User Name: {}".format(user_name))

[INFO] Mode: windows
[INFO] Base Url: http://211.25.204.108:30002/
[INFO] User Name: zong.xian.soh@adv-fusionex.com


In [7]:
# Get web page
instance = TestEngine(mode="windows", driver_path=driver_path, timeout=timeout)
instance.get_driver().get(base_url)

TestEngine: WebDriver created!
Mode: windows
Driver Path: ../../../library/drivers/chrome/win/chromedriver.exe
Timeout: 30


<a id='top'></a>
## Table of contents
- <a href='#data_load'>Load Test Data</a>
- <a href='#login'>Login</a>
- <a href='#switch_company'>Switch Company</a>
- <a href='#user'>User</a>

<a id='login'></a>
### Login
<a href='#top'>top</a>

In [ ]:
ui = UIHelper(instance, user_name=user_name, password=password)
ui.login()

<a id='data_load'></a>
### Load Test Data
<a href='#top'>top</a>

In [ ]:
# Load sample data
data_path =  f"../data/{suite_case}/users.json"
users = ui.load_data(data_path)

<a id='user'></a>
### User
<a href='#top'>top</a>

In [ ]:
ui.click_user_settings("Users")

In [ ]:
for user in users:
    locator = (By.XPATH, f'.//tbody/tr[contains(@class, "row") and .//a[contains(text(), "{user["name"]}")]]')
    elements = instance.find_elements(locator)

    if len(elements) > 0:
        logger.info(f"Found: {user['name']}")
        instance.scroll_to_element(elements[0])
        ui.click_edit(elements[0])
    else:
        logger.info(f"Not Found: {user['name']}")
        ui.click_add()
        
    ui.enter_text((By.ID, "name"), user["name"])
    ui.enter_text((By.ID, "email"), user["email"])
    ui.enter_text((By.ID, "password"), user["password"])
    ui.enter_text((By.ID, "password_confirmation"), user["password"])
    instance.scroll_down()
    ui.select_dropdown("Language", user["language"])
    ui.select_dropdown("Landing Page", user["landing_page"])
    ui.select_dropdown_multiple("Companies", user["companies"])
    for role in user["roles"]:  
        ui.select_toggle_button("Roles", role)
    ui.select_toggle_button("Enabled", user["enabled"])
    ui.click_save()
    instance.wait(2)

In [ ]:
instance.close()